# Predicting House Prices - A case study with SciKit Learn and Pandas

## Introduction to Pandas

Pandas is a module in Python that is great for handling lots of data. We'll be relying on it today to help us sort, reshape, and clean up our data. 

> **From the Pandas documentation:**
>
> Here are just a few of the things that pandas does well:
>
> - Easy handling of **missing data** (represented as NaN) in floating point as well as non-floating point data
- Size mutability: columns can be **inserted and deleted** from DataFrame and higher dimensional objects
- Automatic and explicit **data alignment**: objects can be explicitly aligned to a set of labels, or the user can simply ignore the labels and let Series, DataFrame, etc. automatically align the data for you in computations
- Powerful, flexible **group by** functionality to perform split-apply-combine operations on data sets, for both aggregating and transforming data
- Make it **easy to convert** ragged, differently-indexed data in other Python and NumPy data structures into DataFrame objects
- Intelligent **label-based slicing**, **fancy indexing**, and **subsetting** of large data sets
- Intuitive **merging** and **joining** data sets
- Flexible **reshaping** and **pivoting** of data sets
- **Hierarchical labeling** of axes (possible to have multiple labels per tick)
- **Robust IO tools** for loading data from flat files (CSV and delimited), Excel files, databases, and saving / loading data from the ultrafast HDF5 format
- **Time series**-specific functionality: date range generation and frequency conversion, moving window statistics, moving window linear regressions, date shifting and lagging, etc.

Let's get a sense of how it works with a test data set from a weather report.

In [ ]:
import pandas as pd # the pd is by convention
import numpy as np # as is the np

import matplotlib.pyplot as plt
import seaborn as sns

# To Plot matplotlib figures inline on the notebook
%matplotlib inline

The building blocks of pandas are called Series and DataFrames. A Dataframe is essentially a table, like shown below:

<img src='images/dataframe.png'>

Each row of the dataframe will be one specific record, and each column will be some aspect of that record. That will make more sense when we look at an example. Series are individual rows or columns (essentially if we break apart that dataframe into a single set of numbers). Let's look at that in action.

To begin with, we're going to read in some data from a CSV.

In [ ]:
weather = pd.read_csv('data/weather.csv')
weather.head()

Here we got our table, and it shows us that we're looking at the hourly weather at some point. We can see that the temperature was below freezing for each of the first four hours, that there was some fog, and that it was a little bit windy. We used `weather.head()` to show us just the first few rows. Otherwise, it would show us a HUGE data amount. We can look at the last few rows with `weather.tail()`

In [ ]:
weather.tail()

Now let's use a Pandas built-in function to learn a little about our data.

In [ ]:
weather.info()

We can also look to see how much of our data makes sense by asking it to look at the numeric columns and give us some stats about them.

In [ ]:
weather.describe()

In [ ]:
weather.shape

Great, looks like these are all behaving relatively as expected! That's lovely. Now let's learn how to grab some data from the DataFrame. Also, let's look at what a Series is. To start with, let's grab a column from our data.

In [ ]:
weather['Temp (C)']

This is a series! It has both the index (the left side) and the value. So we know which row it is and what the value is. That's pretty sweet. What can we do with a series? One really handy thing is getting the number of times a value shows up. Let's see that in action.

In [ ]:
weather['Weather'].value_counts()

Cool. Now how do we access rows? We have to play a little bit of pandas games to do so. We'll use `.iloc` to do the job. Let's demonstrate by grabbing the first (0th) row.

In [ ]:
weather.iloc[0]

We can get multiple rows by following Python's conventions like so:

In [ ]:
weather.iloc[10:13]

We also might want to select multiple columns. We can do that like this:

In [ ]:
weather[['Temp (C)',"Dew Point Temp (C)"]].head()

That should be enough to get us started with Pandas... now let's move on to our case study, and we'll learn more about Pandas on the way.

## Building Regression Models with Scikit-Learn

In this section, we will walk through how to build regression models in scikit-learn.

We will load in a the Ames Housing Data, split into train and test sets, and build some models.

Using the Ames Housing Data:

Dean De Cock
Truman State University
Journal of Statistics Education Volume 19, Number 3(2011), www.amstat.org/publications/jse/v19n3/decock.pdf



In [ ]:
datafile = "./data/Ames_Housing_Data.tsv"

In [ ]:
df=pd.read_csv(datafile, sep='\t')

In [ ]:
df.info()

### Data Dictionary
A description of the variables can be found here:

https://ww2.amstat.org/publications/jse/v19n3/decock/DataDocumentation.txt



### Data Cleaning
From the above, and reading the documentation, here are a few things to note about this data set:
- SalePrice is our target variable
- The authors recommend removing the few houses that are >4000 SQFT (based on the 'Gr Liv Area' variable)
- Many columns have missing data (based on the number of "non-null" entries in each column
- We have many predictor variables

** An aside about pandas **

We can do filtering on the fly with pandas. Let's see what that means by looking at our weather data from above.

In [ ]:
weather_test = weather[weather['Temp (C)'] == -1.8]
weather_test.head(10)

We can can pandas to only show us rows where some condition is true. In this case, we asked to only show when the temerature was -1.8 degrees Celsius. We can do the same type of thing with any conditions!

Now back to the housing data!

### Challenge 1: Remove all houses that are greater than 4000 sqft with filtering

- How many data points did we remove from the data set?

In [ ]:
## Next, let's restrict ourselves to just a few variables to get started

In [ ]:
smaller_df= df[['Lot Area','Overall Qual',
       'Overall Cond', 'Year Built', 'Year Remod/Add',
        'Gr Liv Area', 
        'Full Bath', 'Bedroom AbvGr',
        'Fireplaces', 'Garage Cars','SalePrice']]

In [ ]:
## Let's have a look at these variables

smaller_df.describe()

In [ ]:
smaller_df.info()

In [ ]:
# There appears to be one NA in Garage Cars - fill with 0
smaller_df = smaller_df.fillna(0)

In [ ]:
smaller_df.info()

In [ ]:
## Let's do a pairplot with seaborn to get a sense of the variables in this data set
sns.pairplot(smaller_df)

### Comprehension question
From the pairplot above:

- Which variables seem to have the strongest correlations with SalePrice?

In [ ]:
# Let's make a X and y for our predictors and target, respectively

X=smaller_df[['Lot Area','Overall Qual',
       'Overall Cond', 'Year Built', 'Year Remod/Add',
        'Gr Liv Area', 
        'Full Bath', 'Bedroom AbvGr',
        'Fireplaces', 'Garage Cars']]

y=smaller_df['SalePrice']

### Train - Test Splits

Train-test splitting is a big part of the data science pipeline. The reason being, we're always trying to build models that perform well "in the wild." This means that in order to evaluate our model's performance, we need to test it on data that we didn't use when building the model. This means we often want to cut out some section of our data before we do any model-building; to save for use as a "evaluator" of how our model performs on data it's never seen before. 

<img src="train_test_split.png">

In SkLearn, we use `train_test_split` to do this, which allows us to randomly sample the data instead of taking one big chunk. 

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
#Split the data 70-30 train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=42)

In [ ]:
X_train.shape, X_test.shape

In [ ]:
X_train.columns

## Linear Regression
In the first part of this notebook we will use linear regression.  We will start with a simple one-variable linear regression and then proceed to more complicated models.

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
# First let us fit only on Living Area (sqft)
selected_columns_1 = ['Gr Liv Area']

### Sklearn Modeling
The package scikit-learn has a particular structure to their predictive modeling functionality.  Typically, a model is "defined" then it is "fit" (to a set of examples with their answers).  Then the trained model can be used to predict on a set of (unlabeled) data points.  We will walk through this process in the next few cells.

In [ ]:
## First we define a `default` LinearRegression model and fit it to the data (with just `Gr Liv Area' as a predictor
## and SalePrice as the targer.)

lr_model1 = LinearRegression()
lr_model1.fit(X_train[selected_columns_1],y_train)

In [ ]:
## Let us look at the (single) variable coefficient and the intercept
lr_model1.coef_, lr_model1.intercept_

### Comprehension Question
- What would this simple model predict as the sales price of a 1000 sq ft home?
- Does that seem reasonable? (Remember, these are house prices in Ames, Iowa between 2006 and 2010)

## Plotting the Regression Line
Let's use our knowledge of Matplotlib/Seaborn to make some plots of this data

In [ ]:
# TO DO: Make a scatterplot of the sales price (y-axis) vs the sq footage (x-axis) of the training data
# Then, plot the regression line we just computed over the data

plt.figure(figsize=(10,8))
plt.scatter(X_train['Gr Liv Area'],y_train,alpha=.1)
vec1 = np.linspace(0,4000,1000)
plt.plot(vec1, lr_model1.intercept_ + lr_model1.coef_[0]*vec1,'r')
plt.title("Housing Prices in Ames Iowa by Sq Ft (Training Set)")
plt.xlabel("Sq ft of home")
plt.ylabel("Price of home");

In [ ]:
# Let's make a similar plot for the test set

plt.figure(figsize=(10,8))
plt.scatter(X_test['Gr Liv Area'],y_test,alpha=.1)
vec1 = np.linspace(0,4000,1000)
plt.plot(vec1, lr_model1.intercept_ + lr_model1.coef_[0]*vec1,'r')
plt.title("Housing Prices in Ames Iowa by Sq Ft (Test Set)")
plt.xlabel("Sq ft of home")
plt.ylabel("Price of home");



In [ ]:
# Let's get predictions of the model both on the training set and on the test set
# Note the use of the `model.predict(feature_matrix)` syntax

train_set_pred1 = lr_model1.predict(X_train[selected_columns_1])
test_set_pred1 = lr_model1.predict(X_test[selected_columns_1])

In [ ]:
## Let's plot the actual vs expected house price (along with the line x=y for reference)
plt.figure(figsize=(10,8))
plt.scatter(test_set_pred1,y_test,alpha=.1)
plt.plot(np.linspace(0,600000,1000),np.linspace(0,600000,1000))

In [ ]:
# How good is our model on the test set?

# Mean Squared Error
def mean_square_error(true, pred):
    return np.mean((pred - true)**2)

mean_square_error(y_test,test_set_pred1)

In [ ]:
# Root Mean Square Error
def root_mean_square_error(true,pred):
    return np.sqrt(mean_square_error(true,pred))

root_mean_square_error(y_test,test_set_pred1)

In [ ]:
# Mean Absolute Deviation
def mean_absolute_deviation(true,pred):
    return np.mean(np.abs(pred - true))

mean_absolute_deviation(y_test, test_set_pred1)

In [ ]:
# R^2
def R2_score(true,pred):
    y_bar_test = np.mean(true)
    SSE = np.sum((pred - true)**2)
    SST = np.sum((true - y_bar_test)**2)
    return 1.-SSE/SST

R2_score(y_test, test_set_pred1)

In [ ]:
## Now let's use another variable

In [ ]:
selected_columns_2 = ['Lot Area', 'Overall Qual']

In [ ]:
lr_model2 = LinearRegression()
lr_model2.fit(X_train[selected_columns_2],y_train)

In [ ]:
lr_model2.coef_

In [ ]:
## This is a hack to show the variables next to their values
list(zip(selected_columns_2,lr_model2.coef_))

In [ ]:
test_set_pred2 = lr_model2.predict(X_test[selected_columns_2])

In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(test_set_pred2,y_test,alpha=.2)
plt.plot(np.linspace(0,600000,1000),np.linspace(0,600000,1000));

In [ ]:
root_mean_square_error(y_test,test_set_pred2)

In [ ]:
#MAD
mean_absolute_deviation(y_test,test_set_pred2)

In [ ]:
R2_score(y_test,test_set_pred2)

### Feature Engineering
Since there seems to be some non-linearity, let's make a new variable that is "Greater Living Area"^2. This is called feature engineering since we're "engineering (or making)" a new feature out of our old features.

In [ ]:
X['GLA2'] = X['Gr Liv Area']**2
X.columns

In [ ]:
## We need to recreate the train and test sets -- make sure you use the same random seed!
#Split the data 70-30 train/test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=42)

In [ ]:
selected_columns_3 = ['Lot Area', 'Overall Qual', 'GLA2']

In [ ]:
lr_model3 = LinearRegression()
lr_model3.fit(X_train[selected_columns_3],y_train)

In [ ]:
list(zip(X_train[selected_columns_3].columns,lr_model3.coef_))

In [ ]:
test_set_pred3 = lr_model3.predict(X_test[selected_columns_3])

In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(test_set_pred3,y_test,alpha=.1)
plt.plot(np.linspace(0,600000,1000),np.linspace(0,600000,1000));

In [ ]:
#RMSE
root_mean_square_error(y_test,test_set_pred3)

In [ ]:
#MAD
mean_absolute_deviation(y_test,test_set_pred3)

In [ ]:
#R-squared
R2_score(y_test,test_set_pred3)

## Exercise

We're now going to split into groups. Each group should attempt to build the best model they can using the techniques shown above. Some recommendations:

* Add some of the features we removed. But be careful, we haven't talked about how to handle categorical data, so your model won't work with categories.
* Do some feature engineering. We played with GLA^2, but there are more variables you can try things with. You might also try multiplying some features together to see if there are "interaction" terms.
* We've looked at the SkLearn Documentation, so you might also consider trying some different Regression Models - like RandomForestRegressor. Be careful though, you can't just plug-and-play some of the models into the exact same code. They don't all have coefficients for instance...

Go wild. After we finish up, each group will have a chance to describe what sort of work they tried and how their model performed!